In [7]:
pip install patool


Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: python-debian 0.1.43ubuntu1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of python-debian or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pathlib import Path
import zipfile
import os
import shutil
import pyzipper


# 한글 파일명을 올바르게 인코딩
def correct_encoding(filename, encoding='EUC-KR'):
    try:
        return filename.encode('cp437').decode(encoding, 'ignore')
    except UnicodeEncodeError:
        return filename

# 디렉토리 내의 파일과 폴더 이름을 올바르게 한글 인코딩으로 변경
def rename_files_and_folders_in_directory(directory: Path):
    for dirpath, dirnames, filenames in os.walk(directory, topdown=False):
        for filename in filenames:
            correct_filename = correct_encoding(filename)
            if filename != correct_filename:
                shutil.move(os.path.join(dirpath, filename), os.path.join(dirpath, correct_filename))
        for dirname in dirnames:
            correct_dirname = correct_encoding(dirname)
            if dirname != correct_dirname:
                shutil.move(os.path.join(dirpath, dirname), os.path.join(dirpath, correct_dirname))

# 압축 풀기, 파일명 변경, 파일 이동
def unzip_rename_and_move_files(target: str, work_path: str = "/mnt/disks/data1/aihub/Training", remove_current_dir: bool = False):
    work_path = Path(work_path)
    label_files = sorted([file for file in os.listdir(work_path) if file.startswith("[라벨]")])
    source_files = sorted([file for file in os.listdir(work_path) if file.startswith("[원천]")])
    
    target_path = work_path / target
    
    if remove_current_dir and target_path.exists(): # Remove current directory if exists
        shutil.rmtree(target_path)
    elif target_path.exists(): 
        print(f"Directory {target_path} already exists.")
        return
        
    target_path.mkdir(exist_ok=True)
    
    # Unzip label files
    for label in label_files:
        if label.startswith(f"[라벨]{target}"):
            with pyzipper.AESZipFile(work_path / label, 'r') as label_zip:
                label_zip.extractall(target_path)
    
    # Unzip source files
    for source in source_files:
        if source.startswith(f"[원천]{target}"):
            with pyzipper.AESZipFile(work_path / source, 'r') as source_zip:
                source_zip.extractall(target_path)
    
    # Rename files and folders
    rename_files_and_folders_in_directory(target_path)

    # Initialize temp directory to remove
    temp_dirs = os.listdir(target_path)

    # Move files to new location
    dst_path = Path(f"/mnt/disks/data1/aihub/Training/{target}")
    for file in target_path.glob("**/*"):
        if file.is_file():
            relative_path = "/".join(file.relative_to(target_path).parts[1:])
            new_path = dst_path / relative_path
            new_path.parent.mkdir(parents=True, exist_ok=True) 
            shutil.move(str(file), str(new_path))
            
    # Remove temp directory
    for temps in temp_dirs:
        shutil.rmtree(target_path / temps)

In [3]:
target = "서해_대천항_2구역_BOX" # 정리할 데이터셋 이름 (e.g. [라벨]서해_군산항_2구역_BOX.zip, [원천]서해_군산항_2구역_BOX_0 ~ 3.zip 파일이 있다면 "서해_군산항_2구역_BOX")
work_path = "/mnt/disks/data1/aihub/Training" # 압축 파일이 있는 경로
unzip_rename_and_move_files(target, work_path, remove_current_dir=True)